# Imports

In [7]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd


# Load Data

In [ ]:
# Load the data
input_file_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras.xlsx"))[0]
input_file_synergy = list(Path(os.path.join("data")).glob("PTSS_Data_Synergy.xlsx"))[0]
fulltext_foras = list(Path(os.path.join("data")).glob("PTSS_Data_Foras_Fulltext.xlsx"))[
    0
]
fulltext_foras_2nd = list(
    Path(os.path.join("data")).glob("PTSS_Data_Foras_Fulltext_2ndscreener.xlsx")
)[0]
fulltext_synergy = list(
    Path(os.path.join("data")).glob("PTSS_Data_Synergy_Fulltext.xlsx")
)[0]

# Print the file names
print(
    "Results based on file: ",
    input_file_foras,
    input_file_synergy,
    fulltext_foras,
    fulltext_foras_2nd,
    fulltext_synergy,
)

# Read the foras file and filter out the duplicates
foras_unfiltered = pd.read_excel(input_file_foras)
foras_filtered = foras_unfiltered[foras_unfiltered["filter_duplicate"] != 1]

# Read the other files
synergy = pd.read_excel(input_file_synergy)
fulltext_foras = pd.read_excel(fulltext_foras)
fulltext_foras_2nd = pd.read_excel(fulltext_foras_2nd)
fulltext_synergy = pd.read_excel(fulltext_synergy)

# Print the number of rows in each file
print("Number of records in original FORAS file: ", foras_unfiltered.shape[0])
print(
    "Number of records in FORAS after filtering duplicates: ", foras_filtered.shape[0]
)
print("Number of records in SYNERGY", synergy.shape[0])
print("Number of records in FORAS fulltext", fulltext_foras.shape[0])
print("Number of records in FORAS fulltext 2nd screener", fulltext_foras_2nd.shape[0])
print("Number of records in SYNERGY fulltext", fulltext_synergy.shape[0])


# Number of PIDs, Titles and Abstracts

In [ ]:
# Function to calculate missing data and plot for a given dataset
def analyze_missing_data(dataset, dataset_name):
    # Define columns to check for missing values
    columns_to_check = ["doi", "openalex_id", "title", "abstract"]
    missing_data_counts = {col: dataset[col].isnull().sum() for col in columns_to_check}
    missing_data_counts["Both Missing"] = (
        dataset["doi"].isnull().sum() & dataset["openalex_id"].isnull().sum()
    )

    # Print the number of records without certain values
    print(f"Missing Data Analysis for {dataset_name}:")
    for category, count in missing_data_counts.items():
        print(f"  - Number of records without {category}: {count}")

    # Calculate percentages for visualization
    total_records = dataset.shape[0]
    percentages = {
        key: (value / total_records) * 100 for key, value in missing_data_counts.items()
    }

    # Plotting
    plt.figure(figsize=(10, 6))
    categories = list(percentages.keys())
    values = list(percentages.values())
    bars = plt.bar(categories, values, color=plt.cm.tab10(range(len(categories))))

    # Add title and labels
    plt.title(f"Percentage of Missing Data in Each Category ({dataset_name})")
    plt.xlabel("Missing Data Category")
    plt.ylabel("Percentage of Total Records")

    # Annotate bars
    for bar, category in zip(bars, categories):
        height = bar.get_height()
        count = missing_data_counts[category]
        plt.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{count} ({height:.1f}%)",
            ha="center",
            va="bottom",
        )

    # Display the chart
    plt.show()

    # Return results for further programmatic use
    return missing_data_counts


# Call the function for both datasets
foras_results = analyze_missing_data(foras_filtered, "Foras")
synergy_results = analyze_missing_data(synergy, "Synergy")


# Title inclusions